# C2HLSC Tutorial – Leveraging LLMs to refactor C code into HLS‑compatible C

**Goal:** Learn to use **C2HLSC** to transform ordinary C code into *High‑Level Synthesis (HLS)-amenable* C.

**What you'll do:**
1. Set up the environment and clone the C2HLSC repo.
2. Configure LLM API keys (OpenAI, Anthropic, and DeepSeek supported by the repo).
3. Explore C2HLSC CLI options and run a hands‑on example (Cholesky decomposition).
4. Inspect the transformed code and validate its correctness.

## 0) Runtime check
Quick sanity checks to confirm your local toolchain is available.

This tutorial expects:

- **Python 3.11+**

- **gcc**, **g++**, **clang**, and **gdb**

- Access to **Catapult HLS**

If something is missing, install it or run on a machine where it's available.

In [1]:
import platform, sys

print('Python:', sys.version)

print('Platform:', platform.platform())

print('\nGCC version:')
!gcc --version || true

print('\nG++ version:')
!g++ --version || true

print('\nClang version:')
!clang --version || true

print('\nGDB version:')
!gdb --version || true

print('\nCatapult version:')
!catapult -version || true

Python: 3.12.11 (main, Jun 19 2025, 11:41:33) [GCC 8.5.0 20210514 (Red Hat 8.5.0-27)]
Platform: Linux-4.18.0-553.70.1.el8_10.x86_64-x86_64-with-glibc2.28

GCC version:
gcc (GCC) 8.5.0 20210514 (Red Hat 8.5.0-26)
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.


G++ version:
g++ (GCC) 8.5.0 20210514 (Red Hat 8.5.0-26)
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.


Clang version:
clang version 19.1.7 ( 19.1.7-2.module+el8.10.0+23045+e1f8e80e)
Target: x86_64-redhat-linux-gnu
Thread model: posix
InstalledDir: /usr/bin
Configuration file: /etc/clang/x86_64-redhat-linux-gnu-clang.cfg

GDB version:
GNU gdb (GDB) Red Hat Enterprise Linux 8.2-20.el8
Copyright (C) 2018 Free Software Foundatio

## 1) Clone C2HLSC and install Python dependencies
The repo expects Python 3.11+ and a few libraries. We'll install to the current environment.

In [2]:
# Clone the repository

import os, shutil

REPO_URL = 'https://github.com/Lucaz97/c2hlsc'
REPO_DIR = './c2hlsc'

if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone $REPO_URL $REPO_DIR

Cloning into './c2hlsc'...
remote: Enumerating objects: 11389, done.
remote: Counting objects: 100% (2626/2626), done.
remote: Compressing objects: 100% (1192/1192), done.
remote: Total 11389 (delta 1596), reused 2402 (delta 1394), pack-reused 8763 (from 1)
Receiving objects: 100% (11389/11389), 167.50 MiB | 65.99 MiB/s, done.
Resolving deltas: 100% (3771/3771), done.
Updating files: 100% (13354/13354), done.


In [3]:
# Install Python deps expected by the repo

%pip install pycparser openai anthropic pyyaml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## 2) Switch into the C2HLSC repo

From this point onward, all commands and scripts must be run inside the C2HLSC repository directory (`c2hlsc/`).

Make sure you change into the repo root before continuing.

If you restart the notebook later, you’ll need to re-run this cell to set the working directory correctly.

In [4]:
import os

# Path to repo directory (update if you cloned somewhere else)
REPO_DIR = './c2hlsc'

# Change into the repo
os.chdir(REPO_DIR)

print('Now working in:', os.getcwd())

Now working in: /home/ajh9498/Documents/C2HLSC_Tutorial/c2hlsc


## 3) Configure API keys (OpenAI, Anthropic, and/or DeepSeek)
C2HLSC uses an LLM under the hood. Provide at least one key below. Keys are kept in this session only.

In [5]:
import os, getpass

print('Enter API keys when prompted (input hidden). Press Enter to skip any.')

try:
    k = getpass.getpass('OPENAI_API_KEY (optional): ')
    if k.strip(): os.environ['OPENAI_API_KEY'] = k.strip()
except Exception:
    pass

try:
    k = getpass.getpass('ANTHROPIC_API_KEY (optional): ')
    if k.strip(): os.environ['ANTHROPIC_API_KEY'] = k.strip()
except Exception:
    pass

try:
    k = getpass.getpass('DEEPSEEK_API_KEY (optional): ')
    if k.strip(): os.environ['DEEPSEEK_API_KEY'] = k.strip()
except Exception:
    pass

print('Keys set in environment:', [k for k in os.environ if k in ('OPENAI_API_KEY','ANTHROPIC_API_KEY', 'DEEPSEEK_API_KEY')])

Enter API keys when prompted (input hidden). Press Enter to skip any.
Keys set in environment: ['ANTHROPIC_API_KEY', 'DEEPSEEK_API_KEY', 'OPENAI_API_KEY']


## 4) Explore the README and CLI help
The README provides the input structure (includes, functions, test files) and the main entrypoint. Let's open the README and the CLI `-h` to see supported options.

In [6]:
# Show README snippet

!cat README.md || true

# c2hlsc: A framework for automatically refactoring C code into synthesizable C code

### Requirements
The current version of the framework uses Catapult HLS. 
The framework uses gcc, g++ and gdb.

Python dependencies:

    python3.11 -m pip install pycparser openai anthropic pyyaml

An API key for the model in use is needed.
It can be set as an environment variable (ANTHROPIC_API_KEY, OPENAI_API_KEY, DEEPSEEK_API_KEY)

### Input Structure
The C/C++ code should be split into 3 files:

 - includes file: should contain the include libraries  and global variables that should not be refactored. We explicitly prompr the LLM to assume that the content of this file will be provided and should not be produced by the LLM.
 - functions file: should contain all and only  the functions that the LLM should refactor.
 - test file: should contain a main function with one or more tests, the results should be printed to stdout.

One yaml file contains information about top function, file paths, type of

In [7]:
# Show CLI help

!python3.11 src/c2hlsc.py -h || true

usage: c2hlsc.py [-h]
                 [--model {hyperbolic-reasoner,hyperbolic-chat,deepseek-ai/DeepSeek-R1,deepseek-ai/DeepSeek-V3,claude-3-5-sonnet-20240620,claude-3-5-haiku-20241022,gpt-4o-mini,gpt-4-turbo-2024-04-09,gpt-3.5-turbo-0125,gpt-4o,adaptive,o3-mini,deepseek-chat,deepseek-reasoner}]
                 [--opt_target {throughput,latency}]
                 [--characterize CHARACTERIZE] [--opt_runs OPT_RUNS]
                 [--from_saved FROM_SAVED]
                 config

C2HLSC script, yaml config required

positional arguments:
  config                yaml config file with the following fields: orig_code,
                        test_code, includes, tcl, top_function

options:
  -h, --help            show this help message and exit
  --model {hyperbolic-reasoner,hyperbolic-chat,deepseek-ai/DeepSeek-R1,deepseek-ai/DeepSeek-V3,claude-3-5-sonnet-20240620,claude-3-5-haiku-20241022,gpt-4o-mini,gpt-4-turbo-2024-04-09,gpt-3.5-turbo-0125,gpt-4o,adaptive,o3-mini,deepseek-chat,deeps

## 5) Create a hands‑on example: Cholesky decomposition
We'll prepare a minimal input folder under `inputs/cholesky_demo/` with:

- `includes.h` – system includes and any global constants not to be refactored

- `functions.c` – **the code to refactor/optimize**

- `test.c` – a `main()` function that compiles/runs in software to check functional correctness

- `config_cholesky_demo.yaml` – tells C2HLSC where files live and what to transform

Then we'll run C2HLSC with your chosen model and an optimization target (`latency` or `throughput`).

In [11]:
# Split code into includes, functions, and test files

import textwrap, pathlib
base = pathlib.Path('inputs/cholesky_demo')
base.mkdir(parents=True, exist_ok=True)


(base / 'includes.h').write_text(textwrap.dedent('''\


#include <stdio.h>

#ifndef N
#define N 3
#endif

#ifndef TYPE_T
#define TYPE_T int
#endif
typedef TYPE_T type_t;

void cholesky(type_t A[N][N], type_t L[N][N], type_t D[N][N]);


'''))


(base / 'functions.cpp').write_text(textwrap.dedent('''\


void cholesky(type_t A[N][N], type_t L[N][N], type_t D[N][N]) {
    
    for (int i = 0; i < N; i++) {

        for (int j = 0; j <= i; j++) {

            type_t sum = 0;
            for (int k = 0; k < j; k++) {
                // sum.add(sum, L[i][k] * L[j][k] * D[k][k]);
                sum += L[i][k] * L[j][k] * D[k][k];
            }

            if (i == j) { // diagonal element
                L[i][j] = 1;
                // D[i][j].sub(A[i][i], sum);
                D[i][j] = A[i][i] - sum;
            }
            
            else { // off-diagonal element
                type_t numer;
                // numer.sub(A[i][j], sum);
                numer = A[i][j] - sum;
                L[i][j] = (type_t) numer / D[j][j];
            }
        }
    }
}


'''))


(base / 'test.cpp').write_text(textwrap.dedent('''\


int main(void) {

    /* A is the symmetric, positive-definite matrix */
    /* that we want to decompose */
    type_t A[N][N] = { {4, 12, -16}, 
                    {12, 37, -43}, 
                    {-16, -43, 98}
                };
    
    /* L is the lower triangular matrix */
    type_t L[N][N] = {0, 0, 0, 
                    0, 0, 0, 
                    0, 0, 0};

    /* D is the diagonal matrix */
    type_t D[N][N] = {0, 0, 0, 
                    0, 0, 0, 
                    0, 0, 0};

    cholesky(A, L, D);

    printf("L:\\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", L[i][j]);
        }
        printf("\\n");
    }
    printf("D:\\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", D[i][j]);
        }
        printf("\\n");
    }

    return 0;
}


'''))

879

In [12]:
# Create yaml file

import yaml

# A minimal config based on the README description.
config = {
    'tcl': 'inputs/directives.tcl',
    'includes': str(base / 'includes.h'),
    'orig_code': str(base / 'functions.cpp'),
    'test_code': str(base / 'test.cpp'),
    'top_function': 'cholesky',
    'hierarchical': True,
    'opt_constraint': 'area',
    'opt_constraint_tgt': 10000
}
(base / 'config_cholesky_demo.yaml').write_text(yaml.safe_dump(config, sort_keys=False))

print('Created inputs in', base)
!ls -la $base
print('\nConfig:')
print((base / 'config_cholesky_demo.yaml').read_text())

Created inputs in inputs/cholesky_demo
total 20
drwxrwxr-x  2 ajh9498 ajh9498  114 Aug 27 11:12 .
drwxrwxr-x 28 ajh9498 ajh9498 4096 Aug 27 11:12 ..
-rw-rw-r--  1 ajh9498 ajh9498  245 Aug 27 11:14 config_cholesky_demo.yaml
-rw-rw-r--  1 ajh9498 ajh9498  760 Aug 27 11:14 functions.cpp
-rw-rw-r--  1 ajh9498 ajh9498  182 Aug 27 11:14 includes.h
-rw-rw-r--  1 ajh9498 ajh9498  879 Aug 27 11:14 test.cpp

Config:
tcl: inputs/directives.tcl
includes: inputs/cholesky_demo/includes.h
orig_code: inputs/cholesky_demo/functions.cpp
test_code: inputs/cholesky_demo/test.cpp
top_function: cholesky
hierarchical: true
opt_constraint: area
opt_constraint_tgt: 10000



## 6) Choose model and run C2HLSC
Use `-h` output above to see accepted model names (e.g., OpenAI, Anthropic, and DeepSeek variants). Then pick an optimization target: `latency` or `throughput`.

In [13]:
# Run C2HLSC with chosen LLM model and optimization target

MODEL = 'o3-mini' # change to a supported model printed by -h, e.g., 'anthropic:claude-3-5-sonnet'
OPT_TARGET = 'latency' # or 'throughput'
CFG = (base / 'config_cholesky_demo.yaml')

print('Running C2HLSC... this will call the LLM and may take a few minutes depending on your quota.')
!python3.11 src/c2hlsc.py $CFG --model $MODEL --opt_target $OPT_TARGET || true

Running C2HLSC... this will call the LLM and may take a few minutes depending on your quota.
Model:  o3-mini
Running in mode:  standard Hierarchical:  True
Optimization target:  latency
{'cholesky': [], 'main': ['cholesky', 'printf', 'printf', 'printf', 'printf', 'printf', 'printf']}
Building unit test for  cholesky
clang -ggdb -g3 -O0 -fsanitize=address tmp_cholesky/cholesky_complete.c -o tmp_cholesky/to_debug
{'cholesky': [(ArrayDecl(type=ArrayDecl(type=TypeDecl(declname='A',
                                       quals=[
                                             ],
                                       align=None,
                                       type=IdentifierType(names=['type_t'
                                                                 ]
                                                           )
                                       ),
                         dim=Constant(type='int',
                                      value='3'
                            

## 7) Inspect outputs
After running C2HLSC, you should find an `outputs_{kernel}_{model}_{run}/` directory in the repo containing:

- Refactored C code &rarr; `{kernel}_result.c`

- Results log &rarr; `{kernel}.log`

Let’s preview the latest results.

In [ ]:
from pathlib import Path

KERNEL = 'cholesky' # the top function name used above
MODEL = 'o3-mini' # ensure this matches the model used above
RUN = '1' # the run number, adjust if needed

# Point to your latest run directory
run_dir = Path(f'outputs_{KERNEL}_{MODEL}_{RUN}')

# Files to preview
c_file = run_dir / f'{KERNEL}_result.c'
log_file = run_dir / f'{KERNEL}.log'

# Helper function to preview a file
def preview(path):
    print(f"\n=== {path.name} ===")
    try:
        with open(path) as f:
            for line in f:
                print(line.rstrip())

    except FileNotFoundError:
        print(f"[Missing: {path}]")

In [15]:
# Show the refactored C

preview(c_file)


=== cholesky_result.c ===

#include "../include/ac_float.h"
#include "../include/ac_fixed.h"
#include <stdint.h>


#include <stdio.h>

#ifndef N
#define N 3
#endif

#ifndef TYPE_T
#define TYPE_T int
#endif
typedef TYPE_T type_t;

void cholesky(type_t A[N][N], type_t L[N][N], type_t D[N][N]);



void cholesky(type_t A[3][3], type_t L[3][3], type_t D[3][3])
{
  // For a fixed-size (3x3) matrix, fully unrolling all loops minimizes latency by
  // computing all operations concurrently. This increases area but minimizes the critical path.
  #pragma hls_unroll yes
  for (int i = 0; i < 3; i++)
  {
    #pragma hls_unroll yes
    for (int j = 0; j <= i; j++)
    {
      type_t sum = 0;
      #pragma hls_unroll yes
      for (int k = 0; k < j; k++)
      {
        sum += (L[i][k] * L[j][k]) * D[k][k];
      }

      if (i == j)
      {
        L[i][j] = 1;
        D[i][j] = A[i][i] - sum;
      }
      else
      {
        type_t numer = A[i][j] - sum;
        L[i][j] = numer / D[j][j];
      

In [16]:
# Show the log

preview(log_file)


=== cholesky.log ===
o3-mini runs: 7
o3-mini input tokens: 9955
o3-mini output tokens: 13560
# of functions: 1
HLS runs: 7
Compile runs: 10
Time for c2hlsc:  270255
Time for agent:  43588
Agent sequence:  ['synthesis: cholesky 2', 'solution: cholesky 2']
Agent synthesis calls:  1
Agent python calls:  0
Agent profile calls:  0
Agent inspect calls:  0
Agent solution calls:  1
Seconds lost due to API down:  0
  Process        Real Operation(s) count Latency Throughput Reset Length II Comments

  -------------- ----------------------- ------- ---------- ------------ -- --------

  /cholesky/core                      27       8         10            0  0

  Design Total:                       27       8         10            0  0

                     Post-Scheduling    Post-DP & FSM  Post-Assignment

  ----------------- ---------------- ---------------- ----------------

  Total Area Score:   26618.0          72683.7          26431.3

  Total Reg:           6945.8  (26%)    3340.8   (5%) 

## 8) Validate with `g++` (software test)
We’ll compile and execute the transformed C to sanity-check functional correctness outside of HLS. 

Because the transformed file contains a `main()`, we compile and run it directly.

In [ ]:
import subprocess
from pathlib import Path

KERNEL = 'cholesky' # the top function name used above
MODEL = 'o3-mini' # ensure this matches the model used above
RUN = '1' # the run number, adjust if needed

# Point to your latest run directory
run_dir = Path(f'outputs_{KERNEL}_{MODEL}_{RUN}')

c_file = run_dir / f'{KERNEL}_result.c'
exe_file = run_dir / f'{KERNEL}.exe'

# Compile
print(f'Compiling {c_file}...')
subprocess.run(['g++', str(c_file), '-o', str(exe_file)], check=True)

# Run
print(f'\nRunning {exe_file}...\n')
subprocess.run([str(exe_file)])

print('\nDone.')

Compiling outputs_cholesky_o3-mini_2/cholesky_result.c...

Running outputs_cholesky_o3-mini_2/cholesky.exe...

4 12 -16 12 37 -43 -16 -43 98 
1 0 0 3 1 0 -4 5 1 
4 0 0 0 1 0 0 0 9 

Done.
